<a href="https://colab.research.google.com/github/branjbar/nlp-bert-family/blob/master/pytorch/pytorch_roberta_tpu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# CHANGES:
## lr increase
## TRAIN_BATCH_SIZE
## VALID_BATCH_SIZE

In [3]:
# To avoid notebook get disconnected
"""
function ClickConnect(){
console.log("Working"); 
document.querySelector("colab-toolbar-button").click() 
}setInterval(ClickConnect,60000)
"""

"""
caffeinate
"""

'\ncaffeinate\n'

In [0]:
# TODOS:
## add tic and toc to calculate the total time!
## check the pretrained model
## remove neutral from the filtering

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [6]:
# Installs PyTorch, PyTorch/XLA, and Torchvision
# Copy this cell into your own notebooks to use PyTorch on Cloud TPUs 
# Warning: this may take a couple minutes to run

import os
assert os.environ['COLAB_TPU_ADDR'], 'Make sure to select TPU from Edit > Notebook settings > Hardware accelerator'

VERSION = "20200325"  # @ param ["1.5" , "20200325", "nightly"]
!curl https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
!python pytorch-xla-env-setup.py --version $VERSION

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  3727  100  3727    0     0  17096      0 --:--:-- --:--:-- --:--:-- 17096
Updating TPU and VM. This may take around 2 minutes.
Updating TPU runtime to pytorch-dev20200325 ...
Uninstalling torch-1.5.0+cu101:
Done updating TPU runtime: <Response [200]>
  Successfully uninstalled torch-1.5.0+cu101
Uninstalling torchvision-0.6.0+cu101:
  Successfully uninstalled torchvision-0.6.0+cu101
Copying gs://tpu-pytorch/wheels/torch-nightly+20200325-cp36-cp36m-linux_x86_64.whl...
- [1 files][ 83.4 MiB/ 83.4 MiB]                                                
Operation completed over 1 objects/83.4 MiB.                                     
Copying gs://tpu-pytorch/wheels/torch_xla-nightly+20200325-cp36-cp36m-linux_x86_64.whl...
- [1 files][114.5 MiB/114.5 MiB]                                                
Operation completed over 1 objec

In [0]:
import torch
import torch_xla
import torch_xla.core.xla_model as xm
import torch_xla.distributed.xla_multiprocessing as xmp
import torch_xla.distributed.parallel_loader as pl

In [8]:
!pip install transformers
import numpy as np
import pandas as pd
import json
import os
import tokenizers
import string
import transformers
from torch.nn import functional as F
from tqdm import tqdm
import re
from sklearn import model_selection
from transformers import AdamW
from transformers import get_linear_schedule_with_warmup
import time

     |████████████████████████████████| 573kB 3.4MB/s 
     |████████████████████████████████| 1.0MB 18.1MB/s 
     |████████████████████████████████| 890kB 21.2MB/s 
     |████████████████████████████████| 3.7MB 29.7MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.41-cp36-none-any.whl size=893334 sha256=5f2773a8984fdbef96fc9abb20a772d35f4d249945ac7046b9fb5e81c79e751e
  Stored in directory: /root/.cache/pip/wheels/22/5a/d4/b020a81249de7dc63758a34222feaa668dbe8ebfe9170cc9b1
Successfully built sacremoses


In [0]:
KAGGLE_INPUT_PATH = "../input/"
COLAB_INPUT_PATH = "/content/drive/My Drive/input/"
INPUT_PATH = COLAB_INPUT_PATH

DATA_PATH = INPUT_PATH + "tweet-sentiment-extraction/"
CHECKPOINT_PATH = INPUT_PATH + "model_checkpoint/roberta/"
CHECKPOINT_PATH_TORCH = INPUT_PATH + "model_checkpoint/roberta/torch/"
CHECKPOINT_PATH_HP_TUNING = INPUT_PATH + "model_checkpoint/roberta/hp-tuning/"

# Configuration

In [44]:
class Config():
    def __init__(self, config_path=None):

        if config_path:
            with open(config_path,'r') as json_file:
                self.__dict__ = json.load(json_file)
        else:
            
            self.version='tpu-v3'

            # preprocessing
            self.max_len = 192
            self.tweet_size_to_ignore = 1

            # learning
            self.model_path = "models/roberta-base"
            self.model_arch = "abhishek"
            self.drop_out_1 = .1
            self.lr = 3e-5
            self.adamw_weight_decay = 0
            self.use_scheduler = True
            self.scheduler_weight_decay_1=0.00001
            self.scheduler_weight_decay_2=0
            self.label_smoothing=False #https://stackoverflow.com/questions/55681502/label-smoothing-in-pytorch
            # running
            self.train_batch_size = 8
            self.valid_batch_size = 16
            self.num_epochs = 3
            self.num_folds = 5

            # dev
            self.num_folds_to_train = 5
            self.dev = True
        
        print(str(self.__dict__))

    def get(self, var):
        return getattr(self, var, None)


config_file_name = "config-json-tpu-v28.txt"
config_file_path = os.path.join(CHECKPOINT_PATH_HP_TUNING,config_file_name)
if os.path.exists(config_file_path):
    config = Config(config_file_path)
else:
    print("config file not found! Default configuration will be used")
    config = Config()

{'version': 'tpu-v28', 'max_len': 192, 'tweet_size_to_ignore': 1, 'model_path': 'models/roberta-twitter3m', 'model_arch': 'abhishek', 'drop_out_1': 0.3, 'lr': 5e-05, 'adam_type': 'type_1', 'use_scheduler': True, 'scheduler_weight_decay_1': 0.01, 'scheduler_weight_decay_2': 0, 'smoothing_coeff': 0.25, 'gradient_clipping': True, 'max_grad_norm': 0.1, 'train_batch_size': 8, 'valid_batch_size': 16, 'num_epochs': 3, 'num_folds': 5, 'num_folds_to_train': 1, 'dev': False, 'use_epoch_scheduler': False, 'file_name': 'config-json-tpu-v??.txt'}


# Define functions

In [0]:
byte_level_tokenizer = tokenizers.ByteLevelBPETokenizer(
    vocab_file=os.path.join(INPUT_PATH,config.model_path,"vocab.json"), 
    merges_file=os.path.join(INPUT_PATH,config.model_path,"merges.txt"),
    lowercase=True,
    add_prefix_space=True
)

def print_to_file(text, file_path=CHECKPOINT_PATH_TORCH, extension="txt",display=True,file_name=None):
    """
    savs the input into a file affter adding time and version information
    """
    if file_name:
        full_file_path = os.path.join(file_path, file_name)
    else:
        full_file_path = os.path.join(file_path, "%s-logs.%s"%(config.version,extension))
    f = open(full_file_path, 'a')
    

    if type(text) == list:
        text = "\t".join([str(s) for s in text])
    if type(text) != str:
        text = str(text)

    if display:
        print(text)

    f.write(time.strftime("%Y%m%d-%H%M%S"))
    f.write("\t")
    f.write(config.version)
    f.write("\t")
    f.write(text)
    f.write("\n")


class Timer(object):
    """
    a simple class to keep track of running time in a block.
    code example:
    with Timer():
        your code
        your code

    """
    def __init__(self, name=None):
        self.name = name

    def __enter__(self):
        self.tstart = time.time()

    def __exit__(self, type, value, traceback):
        if self.name:
            print_to_file('[%s]' % self.name,)
        print_to_file('Elapsed: %s' % (time.time() - self.tstart))

def jaccard(str1, str2): 

    a = set(str(str1).lower().split()) 
    b = set(str(str2).lower().split())

    c = a.intersection(b)

    if (len(a) + len(b) - len(c)) > 0:
      return float(len(c)) / (len(a) + len(b) - len(c))
    else:
      return 0
      
class TweetModel(transformers.BertPreTrainedModel):
    def __init__(self, conf):
        super(TweetModel, self).__init__(conf)
        self.roberta = transformers.RobertaModel.from_pretrained(os.path.join(INPUT_PATH,config.model_path), config=conf)
        
        if config.model_arch == "abhishek":
            self.drop_out = torch.nn.Dropout(config.drop_out_1)
            self.l0 = torch.nn.Linear(768 * 2, 2)
            torch.nn.init.normal_(self.l0.weight, std=0.02)
        
        if config.model_arch == "abhishek_plus":
            self.drop_out_1 = torch.nn.Dropout(config.drop_out_1)
            self.dense_1 = torch.nn.Linear(768 * 2, 128)
            self.bach_norm = torch.nn.BatchNorm1d(128)
            self.relu = torch.nn.ReLU()
            self.dense_2 = torch.nn.Linear(128, 2)
            torch.nn.init.normal_(self.dense_1.weight, std=0.02)
            torch.nn.init.normal_(self.dense_2.weight, std=0.02)

        if config.model_arch == "karim":
            self.drop_out_1 = torch.nn.Dropout(config.drop_out_1)
            self.conv1d1_1  = torch.nn.Conv1d(in_channels=768*2, out_channels=128, kernel_size=2)
            self.lrelu_1    = torch.nn.LeakyReLU()
            self.conv1d2_1  = torch.nn.Conv1d(in_channels=128, out_channels=64, kernel_size=2)
            self.dense_1    = torch.nn.Linear(in_features=64,out_features=1)
            self.flatten_1  = torch.nn.Flatten()
            self.softmax_1  = torch.nn.Softmax()
            
            self.drop_out_2 = torch.nn.Dropout(config.drop_out_1)
            self.conv1d1_2  = torch.nn.Conv1d(in_channels=768*2, out_channels=128, kernel_size=2)
            self.lrelu_2    = torch.nn.LeakyReLU()
            self.conv1d2_2  = torch.nn.Conv1d(in_channels=128, out_channels=64, kernel_size=2)
            self.dense_2    = torch.nn.Linear(in_features=64,out_features=1)
            self.flatten_2  = torch.nn.Flatten()
            self.softmax_2  = torch.nn.Softmax()

            # initialize weights 
            torch.nn.init.xavier_uniform_(self.conv1d1_1.weight)
            torch.nn.init.xavier_uniform_(self.conv1d2_1.weight)
            torch.nn.init.xavier_uniform_(self.dense_1.weight)
            torch.nn.init.xavier_uniform_(self.conv1d1_2.weight)
            torch.nn.init.xavier_uniform_(self.conv1d2_2.weight)
            torch.nn.init.xavier_uniform_(self.dense_2.weight)
            
            # self.dense_1.bias.fill_(0.01)
            # self.dense_2.bias.fill_(0.01)
            # self.conv1d1_1.bias.fill_(0.01)
            # self.conv1d2_1.bias.fill_(0.01)
            # self.conv1d1_2.bias.fill_(0.01)
            # self.conv1d2_2.bias.fill_(0.01)

    def forward(self, ids, mask, token_type_ids):
        _, _, out = self.roberta(
            ids,
            attention_mask=mask,
            token_type_ids=token_type_ids
        )

        if config.model_arch == "abhishek":
            out = torch.cat((out[-1], out[-2]), dim=-1)
            out = self.drop_out(out)
            logits = self.l0(out)
            start_logits, end_logits = logits.split(1, dim=-1)

        if config.model_arch == "abhishek_plus":
            x = torch.cat((out[-1], out[-2]), dim=-1)
            x = self.drop_out_1(x)
            x = self.dense_1(x)
            # x = self.bach_norm(x)
            x = self.relu(x)
            logits = self.dense_2(x)
            start_logits, end_logits = logits.split(1, dim=-1)

        if config.model_arch == "karim":
            x = torch.cat((out[-1], out[-2]), dim=-1)
            
            x1 = self.drop_out_1(x)
            x1 = x1.transpose(1,2)
            x1 = self.conv1d1_1(x1)
            x1 = self.lrelu_1(x1)
            x1 = self.conv1d2_1(x1)
            x1 = x1.transpose(1,2)
            x1 = self.dense_1(x1)
            x1 = self.flatten_1(x1)
            x1 = self.softmax_1(x1)

            x2 = self.drop_out_2(x)
            x2 = x2.transpose(1,2)
            x2 = self.conv1d1_2(x2)
            x2 = self.lrelu_2(x2)
            x2 = self.conv1d2_2(x2)
            x2 = x2.transpose(1,2)
            x2 = self.dense_2(x2)
            x2 = self.flatten_2(x2)
            x2 = self.softmax_2(x2)

            start_logits = x1
            end_logits = x2


        start_logits = start_logits.squeeze(-1)
        end_logits = end_logits.squeeze(-1)

        return start_logits, end_logits

def process_data(tweet, selected_text, sentiment, tokenizer, max_len):
    tweet = " " + " ".join(str(tweet).split())
    selected_text = " " + " ".join(str(selected_text).split())

    len_st = len(selected_text) - 1
    idx0 = None
    idx1 = None

    for ind in (i for i, e in enumerate(tweet) if e == selected_text[1]):
        if " " + tweet[ind: ind+len_st] == selected_text:
            idx0 = ind
            idx1 = ind + len_st - 1
            break

    char_targets = [0] * len(tweet)
    if idx0 != None and idx1 != None:
        for ct in range(idx0, idx1 + 1):
            char_targets[ct] = 1
    
    tok_tweet = tokenizer.encode(tweet)
    input_ids_orig = tok_tweet.ids
    tweet_offsets = tok_tweet.offsets
    
    target_idx = []
    for j, (offset1, offset2) in enumerate(tweet_offsets):
        if sum(char_targets[offset1: offset2]) > 0:
            target_idx.append(j)
    
    targets_start = target_idx[0]
    targets_end = target_idx[-1]

    sentiment_id = {
        'positive': 1313,
        'negative': 2430,
        'neutral': 7974
    }
    
    input_ids = [0] + [sentiment_id[sentiment]] + [2] + [2] + input_ids_orig + [2]
    token_type_ids = [0, 0, 0, 0] + [0] * (len(input_ids_orig) + 1)
    mask = [1] * len(token_type_ids)
    tweet_offsets = [(0, 0)] * 4 + tweet_offsets + [(0, 0)]
    targets_start += 4
    targets_end += 4

    padding_length = max_len - len(input_ids)
    if padding_length > 0:
        input_ids = input_ids + ([1] * padding_length)
        mask = mask + ([0] * padding_length)
        token_type_ids = token_type_ids + ([0] * padding_length)
        tweet_offsets = tweet_offsets + ([(0, 0)] * padding_length)
    
    return {
        'ids': input_ids,
        'mask': mask,
        'token_type_ids': token_type_ids,
        'targets_start': targets_start,
        'targets_end': targets_end,
        'orig_tweet': tweet,
        'orig_selected': selected_text,
        'sentiment': sentiment,
        'offsets': tweet_offsets
    }


class TweetDataset:
    def __init__(self, tweet, sentiment, selected_text):
        self.tweet = tweet
        self.sentiment = sentiment
        self.selected_text = selected_text
        self.tokenizer = byte_level_tokenizer
        self.max_len = config.max_len
    
    def __len__(self):
        return len(self.tweet)

    def __getitem__(self, item):
        data = process_data(
            self.tweet[item], 
            self.selected_text[item], 
            self.sentiment[item],
            self.tokenizer,
            self.max_len
        )

        return {
            'ids': torch.tensor(data["ids"], dtype=torch.long),
            'mask': torch.tensor(data["mask"], dtype=torch.long),
            'token_type_ids': torch.tensor(data["token_type_ids"], dtype=torch.long),
            'targets_start': torch.tensor(data["targets_start"], dtype=torch.long),
            'targets_end': torch.tensor(data["targets_end"], dtype=torch.long),
            'orig_tweet': data["orig_tweet"],
            'orig_selected': data["orig_selected"],
            'sentiment': data["sentiment"],
            'offsets': torch.tensor(data["offsets"], dtype=torch.long)
        }
def calculate_jaccard_score(
    original_tweet, 
    target_string, 
    sentiment_val, 
    idx_start, 
    idx_end, 
    offsets,
    verbose=False):
    
    if idx_end < idx_start:
        idx_end = idx_start
    
    filtered_output  = ""
    for ix in range(idx_start, idx_end + 1):
        filtered_output += original_tweet[offsets[ix][0]: offsets[ix][1]]
        if (ix+1) < len(offsets) and offsets[ix][1] < offsets[ix+1][0]:
            filtered_output += " "

    if len(original_tweet.split()) <= config.tweet_size_to_ignore:
        filtered_output = original_tweet

    if sentiment_val != "neutral" and verbose == True:
        if filtered_output.strip().lower() != target_string.strip().lower():
            print("********************************")
            print(f"Output= {filtered_output.strip()}")
            print(f"Target= {target_string.strip()}")
            print(f"Tweet= {original_tweet.strip()}")
            print("********************************")

    jac = jaccard(target_string.strip(), filtered_output.strip())
    return jac, filtered_output



class LabelSmoothLoss(torch.nn.Module):
    
    def __init__(self, smoothing=0.0):
        super(LabelSmoothLoss, self).__init__()
        self.smoothing = smoothing
    
    def forward(self, input, target):
        log_prob = F.log_softmax(input, dim=-1)
        weight = input.new_ones(input.size()) * \
            self.smoothing / (input.size(-1) - 1.)
        weight.scatter_(-1, target.unsqueeze(-1), (1. - self.smoothing))
        loss = (-weight * log_prob).sum(dim=-1).mean()
        return loss

## Alternative to implement LabelSmoothingCrossEntropy
# def reduce_loss(loss, reduction='mean'):
#     return loss.mean() if reduction=='mean' else loss.sum() if reduction=='sum' else loss


# class LabelSmoothingCrossEntropy(nn.Module):
#     def __init__(self, epsilon:float=0.1, reduction='mean'):
#         super().__init__()
#         self.epsilon = epsilon
#         self.reduction = reduction
    
#     def forward(self, preds, target):
#         n = preds.size()[-1]
#         log_preds = F.log_softmax(preds, dim=-1)
#         loss = reduce_loss(-log_preds.sum(dim=-1), self.reduction)
#         nll = F.nll_loss(log_preds, target, reduction=self.reduction)
#         return linear_combination(loss/n, nll, self.epsilon)

labels_moothing_loss = LabelSmoothLoss(smoothing=config.get("smoothing_coeff"))

def pos_weight(pred_tensor, pos_tensor, neg_weight=1, pos_weight=1):
    # neg_weight for when pred position < target position
    # pos_weight for when pred position > target position
    gap = torch.argmax(pred_tensor, dim=1) - pos_tensor
    gap = gap.type(torch.float32).detach()
    return torch.where(gap < 0, -neg_weight * gap, pos_weight * gap)

def loss_fn(start_logits, end_logits, start_positions, end_positions):

    if config.get("penalty_loss"):
        loss_fct = torch.nn.CrossEntropyLoss(reduce='none') # do reduction later
        
        start_loss = loss_fct(start_logits, start_positions) * pos_weight(start_logits, start_positions, 1, 1)
        end_loss = loss_fct(end_logits, end_positions) * pos_weight(end_logits, end_positions, 1, 1)
        
        start_loss = torch.mean(start_loss)
        end_loss = torch.mean(end_loss)
        
        total_loss = (start_loss + end_loss)
        return total_loss

    if (config.get("smoothing_coeff")) and (config.get("smoothing_coeff")>0):
        start_loss = labels_moothing_loss.forward(start_logits, start_positions)
        end_loss = labels_moothing_loss.forward(end_logits, end_positions)
        total_loss = (start_loss + end_loss)
        return total_loss

    # default loss function
    loss_fct = torch.nn.CrossEntropyLoss()
    start_loss = loss_fct(start_logits, start_positions)
    end_loss = loss_fct(end_logits, end_positions)
    
    total_loss = (start_loss + end_loss)
    return total_loss


def get_best_start_end_idxs(start_logits, end_logits):
    if (not config.get("span_choice")) or (config.get("span_choice") == "basic"):
        idx_start = np.argmax(start_logits)
        idx_end = np.argmax(end_logits)
        return idx_start, idx_end

    if (config.get("span_choice") == "smart_sum"):
        max_len = len(start_logits)
        a = np.tile(start_logits, (max_len, 1))
        b = np.tile(end_logits, (max_len, 1))
        c = np.tril(a + b.T, k=0).T
        c[c == 0] = -1000
        return np.unravel_index(c.argmax(), c.shape)

# Map function

In [0]:
def map_fn(index, flags):
    ## Setup 
    fold_num = flags["fold_num"]
    # Sets a common random seed - both for initialization and ensuring graph is the same
    torch.manual_seed(6666)

    # Acquires the (unique) Cloud TPU core corresponding to this process's index
    device = xm.xla_device()  
    print_to_file(str(xm.xla_real_devices([str(device)])[0]))


    ## Dataloader construction
    
    train_idx = fold_ids[fold_num][0]
    valid_idx = fold_ids[fold_num][1]
    train_df_fold = train_df.loc[train_idx]
    valid_df_fold = train_df.loc[valid_idx]

    num_train_batches_per_core = int(train_df_fold.shape[0]/config.train_batch_size/8)
    num_valid_batches_per_core = int(valid_df_fold.shape[0]/config.valid_batch_size/8)

    train_dataset = TweetDataset(
        tweet=train_df_fold.text.values,
        sentiment=train_df_fold.sentiment.values,
        selected_text=train_df_fold.selected_text.values
    )

    valid_dataset = TweetDataset(
        tweet=valid_df_fold.text.values,
        sentiment=valid_df_fold.sentiment.values,
        selected_text=valid_df_fold.selected_text.values
    )

    train_sampler = torch.utils.data.distributed.DistributedSampler(
    train_dataset,
    num_replicas=xm.xrt_world_size(),
    rank=xm.get_ordinal(),
    shuffle=True)

    test_sampler = torch.utils.data.distributed.DistributedSampler(
    valid_dataset,
    num_replicas=xm.xrt_world_size(),
    rank=xm.get_ordinal(),
    shuffle=False)

    # Creates dataloaders, which load data in batches
    # Note: test loader is not shuffled or sampled
    train_loader = torch.utils.data.DataLoader(
        train_dataset,
        batch_size=config.train_batch_size,
        sampler=train_sampler,
        num_workers=8,
        drop_last=True)

    test_loader = torch.utils.data.DataLoader(
        valid_dataset,
        batch_size=config.valid_batch_size,
        sampler=test_sampler,
        shuffle=False,
        num_workers=8,
        drop_last=True)


  ## Network, optimizer, and loss function creation

  #  Note: each process has its own identical copy of the model
  #  Even though each model is created independently, they're also
  #  created in the same way.

    model_config = transformers.RobertaConfig.from_pretrained(os.path.join(INPUT_PATH,config.model_path))
    model_config.output_hidden_states = True
    model = TweetModel(conf=model_config).to(device)


    if config.get('adam_type')=="simple_adam":
        optimizer = torch.optim.Adam(model.parameters(), lr=config.lr)
    if config.get('adam_type')=="type_2":
        optimizer = AdamW(model.parameters(), lr=config.lr, correct_bias=False)
    if (not config.get('adam_type')) or (config.get('adam_type')=="type_1"):
        param_optimizer = list(model.named_parameters())
        no_decay = ["bias", "LayerNorm.bias", "LayerNorm.weight"]
        optimizer_parameters = [
            {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': config.scheduler_weight_decay_1},
            {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': config.scheduler_weight_decay_2},
        ]
        optimizer = AdamW(optimizer_parameters, lr=config.lr)
    
    if not config.get("use_epoch_scheduler"):
        scheduler = get_linear_schedule_with_warmup(
            optimizer, 
            num_warmup_steps=0, 
            num_training_steps=int(train_df_fold.shape[0] / config.train_batch_size * config.num_epochs / 8) #TODO: shall we divide by 8TPUs
        )
    else:
        scheduler = get_linear_schedule_with_warmup(
            optimizer, 
            num_warmup_steps=0, 
            num_training_steps=int(config.num_epochs) #TODO: shall we divide by 8TPUs
        )
    ## Trains
    train_start = time.time()
    best_score = -10000
    for epoch in range(config.num_epochs):
        if xm.is_master_ordinal():
            print_to_file("Epoch %i" % (epoch+1))
        
        para_train_loader = pl.ParallelLoader(train_loader, [device]).per_device_loader(device)
        if xm.is_master_ordinal():
            batch_set = tqdm(para_train_loader, total=num_train_batches_per_core, position=0, leave=True)
        else: 
            batch_set = para_train_loader
        
        for batch_num, batch in enumerate(batch_set):
            ids = batch["ids"]
            token_type_ids = batch["token_type_ids"]
            mask = batch["mask"]
            targets_start = batch["targets_start"]
            targets_end = batch["targets_end"]
            sentiment = batch["sentiment"]
            orig_selected = batch["orig_selected"]
            orig_tweet = batch["orig_tweet"]
            targets_start = batch["targets_start"]
            targets_end = batch["targets_end"]
            offsets = batch["offsets"]

            model.train()
            outputs_start, outputs_end = model(
                ids=ids,
                mask=mask,
                token_type_ids=token_type_ids,
            )
            loss = loss_fn(outputs_start, outputs_end, targets_start, targets_end)
            optimizer.zero_grad()
            loss.backward()
            if config.get("gradient_clipping"):
                torch.nn.utils.clip_grad_norm_(model.parameters(), config.max_grad_norm) 
            xm.optimizer_step(optimizer)
            if config.use_scheduler and (not config.get("use_epoch_scheduler")):
                scheduler.step()
        xm.save(
            model.state_dict(),
            os.path.join(CHECKPOINT_PATH_TORCH, '%s-roberta-%i-e%i.bin'%(config.version,fold_num,epoch)),
            master_only=True)
        if config.get("use_epoch_scheduler"):
            scheduler.step()
        
        if xm.is_master_ordinal():
            print("%i - lr: %.5f" %(epoch, optimizer.state_dict()["param_groups"][0]["lr"]))
    # elapsed_train_time = time.time() - train_start
    # print_to_file("TPU %i - finished training. Train time was: %.2fmin" % (index, elapsed_train_time/60)) 

        ## Evaluation
        model.eval()
        eval_start = time.time()
        jaccard_scores = []
        valid_data_size = 0
        with torch.no_grad():
            
            para_test_loader = pl.ParallelLoader(test_loader, [device]).per_device_loader(device)
            for batch_num, batch in enumerate(para_test_loader):
                valid_data_size += len(batch["offsets"])
                outputs_start, outputs_end = model(
                    ids=batch["ids"],
                    mask=batch["mask"],
                    token_type_ids=batch["token_type_ids"]
                )
                outputs_start = torch.softmax(outputs_start, dim=1).cpu().detach().numpy()
                outputs_end = torch.softmax(outputs_end, dim=1).cpu().detach().numpy()

                for px in range(len(batch["offsets"])):

                    targets_start = batch["targets_start"] [px]
                    targets_end = batch["targets_end"][px]
                    offsets = batch["offsets"][px]        

                    idx_start, idx_end = get_best_start_end_idxs(outputs_start[px, :], outputs_end[px, :])

                    jac, _ = calculate_jaccard_score(
                        original_tweet=batch["orig_tweet"][px],
                        target_string=batch["orig_selected"][px],
                        sentiment_val=batch["sentiment"][px],
                        idx_start=idx_start,
                        idx_end=idx_end,
                        offsets=batch["offsets"][px]
                    )
                    jaccard_scores.append(jac)
        score = np.mean(jaccard_scores)
        elapsed_eval_time = time.time() - eval_start
        print_to_file("TPU%i - Evaluation time was: %.2fmin, jaccard average: %.5f" % (index, elapsed_eval_time/60, score))
        print_to_file(
            [
             'F%i'%fold_num,
             "E%i"%(epoch),
             "%.5f"%score,
             'TPU%i'%index,
             "#%i"%(valid_data_size),
             ],
             file_path=CHECKPOINT_PATH_HP_TUNING,
             extension="csv",
             display=False)

In [0]:
if config.dev:
    train_df = pd.read_csv(DATA_PATH + 'train.csv').dropna().sample(n=3000, random_state=888).reset_index()
else:
    train_df = pd.read_csv(DATA_PATH + 'train.csv').dropna().reset_index()

fold_ids = []

kfold = model_selection.KFold(n_splits=config.num_folds, shuffle=True, random_state=42)
for fold_num, (train_idx, valid_idx) in enumerate(kfold.split(train_df.text)):
    fold_ids.append((train_idx, valid_idx))

# Run K-Fold Training

In [0]:
import warnings
warnings.filterwarnings('ignore')
print("VERSION: ", config.version)
for fold_num in range(config.num_folds_to_train):
    print_to_file("Starting Training for Fold %i"%(fold_num+1))
    flags = {"fold_num": fold_num}
    xmp.spawn(map_fn, args=(flags,), nprocs=8, start_method='fork')

VERSION:  tpu-v28
Starting Training for Fold 1
TPU:0
TPU:1
TPU:4
TPU:3
TPU:5
TPU:6
TPU:2
TPU:7
Epoch 1


100%|██████████| 343/343 [01:09<00:00,  4.94it/s]


0 - lr: 0.00003
TPU5 - Evaluation time was: 1.07min, jaccard average: 0.70468
TPU1 - Evaluation time was: 1.10min, jaccard average: 0.67076
TPU2 - Evaluation time was: 1.12min, jaccard average: 0.71228
TPU4 - Evaluation time was: 1.15min, jaccard average: 0.70550
TPU7 - Evaluation time was: 1.16min, jaccard average: 0.71628
TPU3 - Evaluation time was: 1.19min, jaccard average: 0.68780
TPU6 - Evaluation time was: 1.25min, jaccard average: 0.71258
TPU0 - Evaluation time was: 1.26min, jaccard average: 0.74106
Epoch 2


100%|██████████| 343/343 [00:57<00:00,  5.95it/s]


1 - lr: 0.00002
TPU5 - Evaluation time was: 1.05min, jaccard average: 0.70863
TPU1 - Evaluation time was: 1.05min, jaccard average: 0.68066
TPU2 - Evaluation time was: 1.09min, jaccard average: 0.72544
TPU4 - Evaluation time was: 1.12min, jaccard average: 0.71508
TPU7 - Evaluation time was: 1.12min, jaccard average: 0.71739
TPU3 - Evaluation time was: 1.18min, jaccard average: 0.69690
TPU6 - Evaluation time was: 1.23min, jaccard average: 0.71542
TPU0 - Evaluation time was: 1.24min, jaccard average: 0.75261
Epoch 3


 11%|█         | 37/343 [00:07<00:50,  6.02it/s]

# Get Statistics

In [37]:
file_path = os.path.join(CHECKPOINT_PATH_HP_TUNING,"%s-logs.csv"%config.version)
hp_df = pd.read_csv(file_path,sep="\t",header=None)
hp_df.columns = ["datetime","version","fold","epoch","score","tpu", "size"]
print(file_path)
print(hp_df.shape)
hp_df.head(10)

/content/drive/My Drive/input/model_checkpoint/roberta/hp-tuning/tpu-v27-logs.csv
(48, 7)


,datetime,version,fold,epoch,score,tpu,size
0,20200503-065530,tpu-v27,F0,E0,0.48229,TPU7,#64
1,20200503-065532,tpu-v27,F0,E0,0.52039,TPU1,#64
2,20200503-065532,tpu-v27,F0,E0,0.50699,TPU5,#64
3,20200503-065533,tpu-v27,F0,E0,0.49503,TPU6,#64
4,20200503-065533,tpu-v27,F0,E0,0.45083,TPU3,#64
5,20200503-065534,tpu-v27,F0,E0,0.54176,TPU4,#64
6,20200503-065534,tpu-v27,F0,E0,0.44039,TPU0,#64
7,20200503-065534,tpu-v27,F0,E0,0.49243,TPU2,#64
8,20200503-065552,tpu-v27,F0,E1,0.57880,TPU1,#64
9,20200503-065552,tpu-v27,F0,E1,0.58364,TPU7,#64


In [35]:
hp_df_agg = hp_df.groupby(["version","fold","epoch"]).score.mean().reset_index()
best_models = []
best_scores = []

for f in hp_df_agg.fold.unique():
    best_epoch = hp_df_agg[hp_df_agg.fold==f].sort_values(by="score", ascending=False).epoch.iloc[0]
    best_score = hp_df_agg[hp_df_agg.fold==f].sort_values(by="score", ascending=False).score.iloc[0]
    best_model = "%s-roberta-%s-e%s.bin"%(config.version,f[1], best_epoch[1])
    print (best_model, f, best_epoch, "%.5f"%best_score)
    best_scores.append(best_score)
    best_models.append(best_model)
print("overall_score: ", np.mean(best_scores))

tpu-v27-roberta-0-e2.bin F0 E2 0.65224
overall_score:  0.652238125


# Logging summary and cleaning up the models

In [36]:
files_to_remove = []
for f in os.listdir(CHECKPOINT_PATH_TORCH):
    if (f.startswith(config.version)) and (f.endswith(".bin")):
        if f in best_models:
            print(f, "good model")
        else:
            print(f, " will be deleted.")
            files_to_remove.append(f)

if len(files_to_remove)>0:
    print("preparing to delete %i files"%len(files_to_remove))
    time.sleep(5)
    for f in files_to_remove:
        os.remove(os.path.join(CHECKPOINT_PATH_TORCH,f))
    print("deletion complete")    

tpu-v27-roberta-0-e0.bin  will be deleted.
tpu-v27-roberta-0-e1.bin  will be deleted.
tpu-v27-roberta-0-e2.bin good model
preparing to delete 2 files
deletion complete


In [0]:
print_to_file(
    [
     config.version,
     np.mean(best_scores),
     len(best_scores)
     ],
     file_path=CHECKPOINT_PATH_HP_TUNING,
     extension="csv",
     display=True,
     file_name="models_summary.csv",
     )